# 趋势分析

<table align="left">
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Fcloudymoma%2Fgcp-playgroud-public%2Frefs%2Fheads%2Fmaster%2FBigQuery%2Fgoogle_trends.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/cloudymoma/gcp-playgroud-public/blob/master/BigQuery/google_trends.ipynb">
      <img width="32px" src="https://upload.wikimedia.org/wikipedia/commons/9/91/Octicons-mark-github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

### 基本分析 美国有哪些 DMA

In [ ]:
%%bigquery

SELECT DISTINCT dma_name, dma_id
FROM `bigquery-public-data.google_trends.top_terms`
ORDER BY dma_name;

### 纽约现在什么最火爆

In [ ]:
from google.cloud import bigquery
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

client = bigquery.Client()

# ---------------------------------------------------------
# 1. SQL 查询 (使用模糊匹配，兼容不同格式)
# ---------------------------------------------------------
sql_ny_rank = """
WITH ny_data AS (
    SELECT
        term,
        score,
        rank,
        refresh_date
    FROM `bigquery-public-data.google_trends.top_terms`
    WHERE
        -- 【核心修改】使用 LIKE 模糊匹配
        -- 这样无论它是 'New York NY' 还是 'New York, NY' 都能查到
        dma_name LIKE 'New York%NY'
)

SELECT
    term,
    score,
    rank,
    refresh_date
FROM ny_data
WHERE
    -- 查找该地区最新的数据日期
    refresh_date = (SELECT MAX(refresh_date) FROM ny_data)
ORDER BY
    score DESC
LIMIT 20
"""

# 获取数据
print("正在查询 New York 地区的最新热搜...")
df_ny = client.query(sql_ny_rank).to_dataframe()

# ---------------------------------------------------------
# 2. 数据诊断与可视化
# ---------------------------------------------------------
print("-" * 30)
if df_ny.empty:
    print("❌ 警告：依然没有数据。")
    print("这说明可能不仅是逗号的问题，可能是以下原因：")
    print("1. 数据集里压根没有纽约的数据（断更）。")
    print("2. 现在的表结构里 dma_name 字段完全变成了另一种写法。")

    # 打印个诊断建议，帮您看看到底叫什么
    print("\n建议运行这句 SQL 看看数据库里到底叫什么：")
    print("SELECT DISTINCT dma_name FROM `bigquery-public-data.google_trends.top_terms` WHERE dma_name LIKE '%York%' LIMIT 5")
else:
    date_str = df_ny['refresh_date'].iloc[0]
    print(f"✅ 数据获取成功！日期: {date_str}")
    print(df_ny[['rank', 'term', 'score']].head())

    # ---------------------------------------------------------
    # 3. 可视化：热搜词排行榜 (Term vs Score)
    # ---------------------------------------------------------
    plt.figure(figsize=(12, 8))

    # X轴 = 热度分数 (Score)
    # Y轴 = 搜索词 (Term)
    sns.barplot(
        data=df_ny,
        x='score',
        y='term',
        palette='Spectral', # 使用醒目的色系
        edgecolor='black'   # 给条形加个边框，更清晰
    )

    plt.title(f'Top 20 Trending Terms in New York ({date_str})', fontsize=16, fontweight='bold')
    plt.xlabel('Search Popularity Score', fontsize=12)
    plt.ylabel('') # 隐藏Y轴标题，词本身很直观

    # 在条形图末尾标注具体分数，看起来更专业
    for index, row in df_ny.iterrows():
        plt.text(row.score + 1, index, f'{int(row.score)}', va='center', fontsize=10)

    plt.grid(axis='x', linestyle='--', alpha=0.3)
    plt.tight_layout()
    plt.show()

### 对比同一话题在“东海岸”与“西海岸”的热度差异

In [ ]:
from google.cloud import bigquery
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

client = bigquery.Client()

# ---------------------------------------------------------
# 参数设置
# ---------------------------------------------------------
# 建议换成一个更通用的词测试逻辑，比如 "NBA", "Weather", "Trump"
# 因为 "minneapolis shooting" 如果不是今天发生的大事，很难同时进入两个城市的 Top 25
target_term = "alan jackson"
target_dmas = ['New York%NY', 'Los Angeles%CA'] # 使用 % 进行模糊匹配，防止逗号问题

# 构造 SQL 里的列表字符串
dma_filters = " OR ".join([f"dma_name LIKE '{dma}'" for dma in target_dmas])

# ---------------------------------------------------------
# SQL 查询
# ---------------------------------------------------------
sql_compare = f"""
SELECT
  dma_name,
  term,
  score,
  rank,
  refresh_date
FROM `bigquery-public-data.google_trends.top_terms`
WHERE
  term like '{target_term}'
  AND ({dma_filters})

  -- 【核心优化】不用全表最新日期，而是取“每个地区自己”的最新日期
  -- 只有最近 14 天的数据才算数，太久以前的没有对比意义
  AND refresh_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 14 DAY)

-- QUALIFY 是 BigQuery 的神技，这里表示：按地区分组，只取日期最新的那一条
QUALIFY ROW_NUMBER() OVER(PARTITION BY dma_name ORDER BY refresh_date DESC) = 1
ORDER BY
  score DESC
"""

# 获取数据
print(f"正在对比关键词 [{target_term}] 在不同地区的表现...")
df_compare = client.query(sql_compare).to_dataframe()

# ---------------------------------------------------------
# 数据诊断与可视化
# ---------------------------------------------------------
print("-" * 30)
if df_compare.empty:
    print("❌ 警告：结果为空。")
    print(f"原因分析：关键词 '{target_term}' 在过去 14 天内，可能未能进入这两个地区的每日 Top 25 榜单。")
    print("建议：换一个更大众的热词（如 'Weather', 'News'）来测试代码逻辑。")
else:
    print(f"✅ 数据获取成功！\n")
    #print(df_compare[['dma_name', 'term', 'score', 'rank']])

    # 绘制对比柱状图
    plt.figure(figsize=(8, 6))

    # 颜色映射：根据地区给不同颜色
    sns.barplot(
        data=df_compare,
        x='dma_name',
        y='score',
        palette='viridis',
        edgecolor='black'
    )

    plt.title(f'Interest Comparison: "{target_term}"', fontsize=14, fontweight='bold')
    plt.ylabel('Search Popularity Score (0-100)', fontsize=12)
    plt.xlabel('Market Area (DMA)', fontsize=12)
    plt.ylim(0, 110) # 固定Y轴最高为110，留出空间显示数字

    # 在柱子上标出具体分数
    for index, row in df_compare.iterrows():
        # 这里需要一点技巧来对应 x 轴的位置，简单起见我们用 enumerate
        plt.text(index, row.score + 2, f'{int(row.score)}', ha='center', fontsize=12, fontweight='bold')

    plt.grid(axis='y', linestyle='--', alpha=0.3)
    plt.tight_layout()
    plt.show()

### 特定话题的生命周期趋势分析 (Time Series Analysis)

过去180天的数据，观察某个特定关键词（或包含特定子串的词，如 "AI", "World Cup"）的排名变化，从而判断该话题是在爆发期、平稳期还是衰退期

In [ ]:
from google.cloud import bigquery
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1. 初始化客户端
client = bigquery.Client()

# 设定参数
keyword = "地震情報"  # 如果没数据，尝试换成更通用的词，如 "Game" 或 "Weather"
lookback_days = 180

# 2. 编写查询 (使用 MAX date 确保有数据锚点)
sql_trend = f"""
WITH max_date_check AS (
    SELECT MAX(refresh_date) as last_date
    FROM `bigquery-public-data.google_trends.international_top_terms`
)
SELECT
  week,
  term,
  score,
  rank
FROM `bigquery-public-data.google_trends.international_top_terms`, max_date_check
WHERE
  country_name like "%apan%"
  -- 动态获取最近半年数据，避免硬编码日期导致无数据
  AND refresh_date >= DATE_SUB(max_date_check.last_date, INTERVAL {lookback_days} DAY)
  AND term LIKE '%{keyword}%'
ORDER BY
  week ASC, rank ASC
"""

# 3. 获取数据
print(f"正在查询关键词: {keyword} ...")
df_trend = client.query(sql_trend).to_dataframe()

# 4. 【关键步骤】数据诊断与打印
print("-" * 30)
print(f"查询结果行数: {len(df_trend)}")

if df_trend.empty:
    print("❌ 警告：未找到数据。")
    print("可能原因：1. 该关键词在所选地区不热门；2. 数据集尚未更新到该日期范围。")
else:
    # print("✅ 数据获取成功，前 5 行数据如下：")
    # print(df_trend.head()) # 打印数据预览

    # 5. 数据转换与可视化
    df_trend['week'] = pd.to_datetime(df_trend['week'])

    plt.figure(figsize=(12, 6))
    sns.lineplot(data=df_trend, x='week', y='score', hue='term', marker='o')
    plt.title(f'Trend Analysis for "{keyword}" (Japan)')
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

### 区域热度对比 (Geographic Heatmap Analysis)

对于美国市场（DMA 数据集）或全球市场，分析同一个热门词在不同地区的表现差异。这对于广告投放定向非常有帮助，可以帮您找出对该话题最感兴趣的城市或地区。

In [ ]:
# 1. 编写查询：不指定词，而是自动寻找当天排名第一的词，分析其地区分布
sql_geo_auto = """
WITH latest_data AS (
  SELECT MAX(refresh_date) as max_date
  FROM `bigquery-public-data.google_trends.top_terms`
),
top_term_finder AS (
    -- 找出全美总分最高的一个词
    SELECT term
    FROM `bigquery-public-data.google_trends.top_terms` t
    JOIN latest_data l ON t.refresh_date = l.max_date
    ORDER BY score DESC
    LIMIT 1
)

SELECT
  t.dma_name,
  t.term,
  t.score
FROM `bigquery-public-data.google_trends.top_terms` t
JOIN latest_data l ON t.refresh_date = l.max_date
JOIN top_term_finder f ON t.term = f.term -- 只查这个最热词的分布
ORDER BY
  t.score DESC
LIMIT 20
"""

# 2. 获取数据
print("正在寻找当前最热词并分析地区分布...")
df_geo = client.query(sql_geo_auto).to_dataframe()

# 3. 【关键步骤】数据诊断与打印
print("-" * 30)
print(f"查询结果行数: {len(df_geo)}")

if df_geo.empty:
    print("❌ 警告：未找到数据。请检查公共数据集连接。")
else:
    target_term = df_geo['term'].iloc[0] # 获取自动找到的那个词
    print(f"✅ 自动锁定最热词: [{target_term}]")
    print("前 5 个地区数据如下：")
    print(df_geo[['dma_name', 'score']].head())

    # 4. 可视化
    plt.figure(figsize=(10, 8))
    sns.barplot(data=df_geo, y='dma_name', x='score', palette='viridis', hue='dma_name', legend=False)
    plt.title(f'Regional Popularity for Top Term: "{target_term}"')
    plt.xlabel('Search Score')
    plt.ylabel('DMA Region')
    plt.show()

### 潜力爆款挖掘 (Rising vs Top Cross-Analysis)

寻找那些出现在 "Rising Terms" (飙升榜) 中，但尚未进入或刚刚进入 "Top Terms" (总榜) 的词。这些词通常代表着正在形成的趋势，是捕捉早期机会的关键。

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# 1. 编写查询
sql_opportunity = """
WITH latest_date AS (
    SELECT MAX(refresh_date) as date FROM `bigquery-public-data.google_trends.top_rising_terms`
),
rising_agg AS (
    SELECT
        term,
        COUNT(DISTINCT dma_name) as rising_region_count
    FROM `bigquery-public-data.google_trends.top_rising_terms` r
    JOIN latest_date l ON r.refresh_date = l.date
    GROUP BY 1
),
top_agg AS (
    SELECT
        term,
        AVG(score) as avg_top_score
    FROM `bigquery-public-data.google_trends.top_terms` t
    JOIN latest_date l ON t.refresh_date = l.date
    GROUP BY 1
)

SELECT
    r.term,
    r.rising_region_count,
    COALESCE(t.avg_top_score, 0) as current_top_score,
    CASE
        WHEN t.avg_top_score IS NULL THEN 'New Breakout (潜力)'
        WHEN t.avg_top_score < 50 THEN 'Growing (成长)'
        ELSE 'Established (热门)'
    END as status
FROM rising_agg r
LEFT JOIN top_agg t ON r.term = t.term
WHERE r.rising_region_count > 5 -- 筛选至少在5个地区飙升的词
ORDER BY rising_region_count DESC, current_top_score ASC
LIMIT 50
"""

# 2. 获取数据
print("正在挖掘潜力爆款词...")
df_opp = client.query(sql_opportunity).to_dataframe()

# 3. 【关键步骤】数据诊断与打印
print("-" * 30)
print(f"查询结果行数: {len(df_opp)}")

if df_opp.empty:
    print("❌ 警告：结果为空。")
    print("尝试建议：将 SQL 中的 'WHERE r.rising_region_count > 5' 条件调低，或者检查 Rising 表是否有最新数据。")
else:
    # 1. 准备数据字典：{词: 飙升地区数}
    # 词云是根据数值大小来决定字体大小的
    data_dict = dict(zip(df_opp['term'], df_opp['rising_region_count']))

    # 2. 生成词云
    wc = WordCloud(
        width=1200,
        height=600,
        background_color='white', # 白色背景更适合文档
        colormap='plasma',        #以此色系为主
        max_words=50,             # 最多显示50个词
        relative_scaling=0.5      # 字号与频率的相关性
    ).generate_from_frequencies(data_dict)

    # 3. 绘图
    plt.figure(figsize=(14, 7))
    plt.imshow(wc, interpolation='bilinear')
    plt.axis('off') # 关闭坐标轴
    plt.title('Rising Trends Word Cloud', fontsize=16, pad=20)
    plt.show()